# Purpose
This file shows the steps taken to load the pre-trained model distilbert and train it with a sample data set

https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb

### Imports

In [1]:
%%capture
!pip install transformers==4.20.0

In [2]:
%%capture
!python -m spacy download de_core_news_lg

In [3]:
%%capture
!pip install datasets

In [4]:

from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import json
import spacy
# prepare
import datasets
from datasets import Dataset
# encode
from transformers import AutoTokenizer
import tensorflow as tf
import torch
# modelling
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
#evaluation 
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction

### Constants

In [5]:
# paths
FILTERED_PATH="filtered_4_26"
ANNOTATED_PATH="annotated"

### Data

In [6]:
# # connect with google drive
# from google.colab import drive
# drive.mount('/content/drive')


In [7]:
# change cwd
%cd drive/MyDrive/Work/Frontline/data

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [9]:
#read json data
json_data=json.load(open(ANNOTATED_PATH+"/annotations_05_08.json", encoding="utf-8"))

### Prepare data

In [10]:
#convert to dataframe
data=pd.DataFrame(json_data["documents"])

In [11]:
#for now: filter out paragraphs that have not been annotated 
data=data[data["annotations"].apply(len)>0]
#reset index
data=data.reset_index(drop=True)
data.head()

,id,text,annotations,attributes_flat
0,4572dea4-6a08-4f1e-b312-5821112bb5f5,Ein Mann (25) ist jetzt vom Schöffengericht am...,"[{'str_start': None, 'str_stop': None, 'annota...",{'artikel_id': 'F3A4578D33A8603DF0573D3DE3CB26...
1,0bcada32-8dc5-41cf-b83b-67d2e742bada,Als Zeugin trat die Ex-Lebensgefährtin des Syr...,"[{'str_start': None, 'str_stop': None, 'annota...",{'artikel_id': 'F3A4578D33A8603DF0573D3DE3CB26...
2,a30791b9-522e-45c1-8b33-79d4165282af,"Zunächst leugnete der Angeklagte, dass es über...","[{'str_start': None, 'str_stop': None, 'annota...",{'artikel_id': 'F3A4578D33A8603DF0573D3DE3CB26...
3,043e3909-bcdd-4c6b-a54f-f947d46ad18e,Das Schöffengericht hatte es in diesem Fall of...,"[{'str_start': None, 'str_stop': None, 'annota...",{'artikel_id': 'F3A4578D33A8603DF0573D3DE3CB26...
4,08cef91c-6d73-472c-8349-07a5b72009d1,"""Gewalt in der Familie ist weder Privatsache n...","[{'str_start': None, 'str_stop': None, 'annota...","{'artikel_id': 'IRA-82182598', 'name': 'SÜDWES..."


In [12]:
data["tags"]=[""]*data.shape[0]

In [13]:
# extract document labels
# ASSUMPTION: all annotations are document labels, ie. do not have start and end as those are ignored
 
for idx, entry in data.iterrows():
  # in case there are more than 1 label
  for label in range(len(entry["annotations"])):
    data.loc[idx, "tags"]+=entry["annotations"][label]["concept"]["preferred_label"]["name"]+"*"

In [14]:
df=data[["tags", "text"]]

In [15]:
# converting the annotation column with one hot encoding
df=pd.merge(df,df["tags"].str.get_dummies(sep="*"), left_index=True, right_index=True)
# dropping the tags column
df=df.drop("tags",1)
df.head()

<ipython-input-15-86d84d902e10>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop("tags",1)


,text,Graphic,NA,Sensationalist,Victim blaming
0,Ein Mann (25) ist jetzt vom Schöffengericht am...,0,1,0,0
1,Als Zeugin trat die Ex-Lebensgefährtin des Syr...,0,0,0,1
2,"Zunächst leugnete der Angeklagte, dass es über...",0,0,0,1
3,Das Schöffengericht hatte es in diesem Fall of...,0,0,1,1
4,"""Gewalt in der Familie ist weder Privatsache n...",0,1,0,0


In [16]:
# create a list of all labels
labels=df.columns.to_list()[1:]

### Preprocessing Data

In [17]:
# needs to be imported after setting path
import preprocessing

In [18]:
# Load model
spacy_mod = spacy.load("de_core_news_lg", disable=['ner', 'parser', 'tagger'])

In [20]:
# read custom stopwords

# open list of custom stopwords
custom_stop_words= open("custom_stopwords.txt", encoding="utf-8").read().split()

# add custom stopwords to model
for word in custom_stop_words:
  spacy_mod.Defaults.stop_words.add(word)

In [21]:
# convert corpus to language object
spacy_lang = []
for i, doc in tqdm(df.iterrows()): 
  spacy_lang.append(spacy_mod("".join(doc['text'])))

124it [00:01, 92.71it/s]


In [22]:
# preprocess: remove stopwords
spacy_cleaned = []
for doc in tqdm(spacy_lang): 
    spacy_cleaned.append(preprocessing.preprocess(doc, remove_ent=True))

100%|██████████| 124/124 [00:00<00:00, 9662.68it/s]


In [ ]:
# replace the text column with the cleaned texts
df["text"]=spacy_cleaned
# convert list of words to string
df["text"]= [" ".join(ls) for ls in df["text"]]

### Split data into train and test

In [23]:
#split train, dev , test sets
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

#### Convert DataFrame to  Dataset

In [24]:
ds=datasets.DatasetDict({"train":Dataset.from_dict(df_train),"val":Dataset.from_dict(df_val),"test":Dataset.from_dict(df_test)})

### Tokenizing & Encoding

In [25]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [26]:
def preprocess_data(df):
  text = df["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  
  labels_batch = {k: df[k] for k in df.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
  encoding["labels"] = labels_matrix.tolist()
  return encoding

In [27]:
ds_encoded=ds.map(preprocess_data, batched=True, remove_columns=ds['train'].column_names)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

### Modelling

In [32]:
# some more expect column name to be "label" not "labels"
ds_encoded["train"]=ds_encoded["train"].rename_column("labels","label")
ds_encoded["test"]=ds_encoded["test"].rename_column("labels","label")
ds_encoded["val"]=ds_encoded["val"].rename_column("labels","label")

In [28]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [29]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'classifie

In [31]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [42]:
args = TrainingArguments(
    f"bert-finetuned-tagging-german",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


#### Training

In [43]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_encoded["train"],
    eval_dataset=ds_encoded["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [44]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 99
  Num Epochs = 10
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 170


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.570033,0.461538,0.626050,0.500000
2,No log,0.599930,0.461538,0.626050,0.500000
3,No log,0.592244,0.461538,0.626050,0.500000
4,No log,0.608316,0.461538,0.626050,0.500000
5,No log,0.586559,0.363636,0.584034,0.333333
6,No log,0.562784,0.315789,0.577731,0.250000
7,No log,0.628944,0.363636,0.584034,0.333333
8,No log,0.618715,0.347826,0.569328,0.333333
9,No log,0.630799,0.363636,0.584034,0.333333
10,No log,0.639780,0.363636,0.584034,0.333333


***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
Saving model checkpoint to bert-finetuned-tagging-german/checkpoint-17
Configuration saved in bert-finetuned-tagging-german/checkpoint-17/config.json
Model weights saved in bert-finetuned-tagging-german/checkpoint-17/pytorch_model.bin
tokenizer config file saved in bert-finetuned-tagging-german/checkpoint-17/tokenizer_config.json
Special tokens file saved in bert-finetuned-tagging-german/checkpoint-17/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
Saving model checkpoint to bert-finetuned-tagging-german/checkpoint-34
Configuration saved in bert-finetuned-tagging-german/checkpoint-34/config.json
Model weights saved in bert-finetuned-tagging-german/checkpoint-34/pytorch_model.bin
tokenizer config file saved in bert-finetuned-tagging-german/checkpoint-34/tokenizer_config.json
Special tokens file saved in bert-finetuned-tagging-german/checkpoint-34/special_tokens_map.json
***** 

TrainOutput(global_step=170, training_loss=0.27982117148006663, metrics={'train_runtime': 129.8417, 'train_samples_per_second': 7.625, 'train_steps_per_second': 1.309, 'total_flos': 32786850539520.0, 'train_loss': 0.27982117148006663, 'epoch': 10.0})

In [45]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 12
  Batch size = 6


{'eval_loss': 0.5700328350067139,
 'eval_f1': 0.4615384615384615,
 'eval_roc_auc': 0.6260504201680672,
 'eval_accuracy': 0.5,
 'eval_runtime': 0.1361,
 'eval_samples_per_second': 88.171,
 'eval_steps_per_second': 14.695,
 'epoch': 10.0}

## Predicting Test Set Labels

In [37]:
sigmoid = torch.nn.Sigmoid()

In [46]:
test_pred=pd.DataFrame(ds["test"])

In [47]:
def predict_text(text):
  encoding = tokenizer(text, return_tensors="pt")
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
  outputs = trainer.model(**encoding)
  logits = outputs.logits
  probs = sigmoid(logits.squeeze().cpu())
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs >= 0.5)] = 1
  # turn predicted id's into actual label names
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
  return predicted_labels
  

In [48]:
pred=[]
for t in ds["test"]["text"]:
  pred.append(predict_text(t))
test_pred["prediction"]=pred

In [49]:
test_pred.head()

,text,Graphic,NA,Sensationalist,Victim blaming,prediction
0,"Als die Frau sich trennen wollte, soll er sie ...",0,0,0,1,[NA]
1,Schnell wird klar: Der Mann hatte die Frau mit...,0,1,0,0,[NA]
2,Nürnberg Auf der Straße sind am Samstag in Nür...,0,1,0,0,[NA]
3,"Wollte er tatsächlich seine Frau nur für ein, ...",0,0,1,1,[NA]
4,Frankfurt/M. Eine Autofahrerin ist bei einer g...,0,1,0,0,[NA]


In [ ]:
accuracy_score